# Spectral Axes: Velocity Frames and Doppler Conventions

This notebook shows how to change spectral axes to represent different rest frames and Doppler conventions.  In addition, a spectrum is normally
associated with a redshift/radial velocity  (PJT?)

Supported doppler conventions:
1. radio
2. relativistic
3. optical

Supported rest frames (some of)
1. ITRF or topocentric
2. ICRF or heliocentric
3. GCRS
4. LSRK, LSRD
5. cmb

This dysh code dealing with this relies heavily on [specutils](https://specutils.readthedocs.io/). To note is that the `Spectrum` class is inherited from the (now) namesake `Spectrum` class in specutils. Internally we still call that class `Spectrum1D`.

See also https://www.gb.nrao.edu/~fghigo/gbtdoc/doppler.html


Although the `Spectrum.plot()` can plot a spectrum in different frames with different conventions, it does not modify the underlying data and meta-data in the spectrum.  To make these persistent (necessary when writing a spectrum) there is both an in-place and copy operation to modify frame and convention. Here's a dysh command summary that we will cover in this notebook, leaving out the values and arguments that are not relevant:

Note that *frame* and *convention* have confusing keywords, this ought to be discussed.

```
       ta.plot(vel_frame=, doppler_convention=)

       ta.velocity_axis_to(toframe=, doppler_convention=)

       ta.set_frame()
       ta.set_convention()

       ta1 = ta.with_frame()
       ta2 = ta1.with_velocity_convention()

       ta.set_redshift.to()
       ta.set_radial_velocity_to()
       ta.shift_spectrum_to(redshift=, radial_velocity=)

       ta.rest_value =
       ta.doppler_rest = 

       # no setters for these
       ta.redshift
       ta.radial_velocity
```


## Loading Modules
We start by loading the modules we will use for the data reduction. 

For display purposes, we use the static (non-interactive) `matplotlib` backend in this tutorial. However, you can tell `matplotlib` to use the `ipympl` backend to enable interactive plots. This is only needed if working on `jupyter` lab or notebook.

In [1]:
# Set interactive plots in jupyter.
#%matplotlib ipympl

# These modules are required for working with the data.
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.log import init_logging
from astropy import units as u
from dysh.spectra.spectrum import Spectrum

# These modules are used for file I/O
from dysh.util.files import dysh_data
from pathlib import Path

## Setup
We start the dysh logging, so we get more information about what is happening.
This is only needed if working on a notebook.
If using the CLI through the ``dysh`` command, then logging is setup for you.

In [2]:
init_logging(2)

# also create a local "output" directory where temporary notebook files can be stored.
output_dir = Path.cwd() / "output"
output_dir.mkdir(exist_ok=True)

## Data Retrieval

Download the example SDFITS data, if necessary.

In [3]:
filename = dysh_data(test="getps")

12:33:57.967 I Resolving test=getps -> AGBT05B_047_01/AGBT05B_047_01.raw.acs/


## Data Loading

In [4]:
sdfits = GBTFITSLoad(filename)
sdfits.summary()

SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATION
51,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,198.3431,18.6427
52,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,198.9306,18.7872
53,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,199.3305,18.3561
54,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,199.9157,18.4927
55,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,200.3042,18.0575
56,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,200.8906,18.1860
57,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,202.3275,17.3853
58,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,202.9192,17.4949


## Data Reduction

Next we fetch and calibrate the position switched data. We will use this data to show how to change rest frames and Doppler conventions. All of these occur at the spectrum level, so we need to get the spectrum first.

In [5]:
psscan = sdfits.getps(scan=51, ifnum=0, plnum=0, fdnum=0)

Create the time-averaged spectrum.

In [6]:
ta = psscan.timeaverage()

## Changing the x-axis of a `Spectrum` Plot
Note this changes the axis of the plot but does not affect the underlying Spectrum object.


### Default Rest Frame

The default plot uses the frequency frame and Doppler convention found in the SDFITS file.
In this case, that is topocentric frame (ITRS) and the optical convention. In the SDFITS file you would find the keyword VELDEF to have the value OPTI-LSR:  (PJT: LSR ?)
```
    print(sdfits['VELDEF'][0])
```

4410.07   topo based radial velocity of target w.r.t. observatory
4386 = VELOCITY   (catalog)  LSR
'RVSYS': 4376,523.966595529,      vsource - vtelescop
'VFRAME': 22,609.232181632025,    rv of velocity frame

In [7]:
print(sdfits['VELDEF'][0])

OPTI-LSR


In [8]:
ta

<Spectrum(flux=[0.24484208600654486 ... 1.2076788454438225] K (shape=(32768,), mean=0.28071 K); spectral_axis=<SpectralAxis 
   (observer: <ITRS Coordinate (obstime=2005-06-27T02:05:58.000, location=(0., 0., 0.) km): (x, y, z) in m
                  (882593.9465029, -4924896.36541728, 3943748.74743984)
               (v_x, v_y, v_z) in km / s
                  (0., 0., 0.)>
    target: <SkyCoord (FK5: equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
                (206.85210758, -30.40701531, 1000000.)
             (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                (0., 0., 4386.)>
    observer to target (computed from above):
      radial_velocity=4410.070039086893 km / s
      redshift=0.014820217767934851
    doppler_rest=1420405000.0 Hz
    doppler_convention=optical)
  [1.42481684e+09 1.42481531e+09 1.42481379e+09 ... 1.37482142e+09
 1.37481989e+09 1.37481836e+09] Hz> (length=32768))>

In [9]:
ta.meta

{'OBJECT': 'NGC5291',
 'BANDWID': 50000000.0,
 'DATE-OBS': '2005-06-27T02:05:58.00',
 'DURATION': 55.5225,
 'EXPOSURE': 53.7157844463702,
 'TSYS': 19.353858908450768,
 'TDIM7': '(32768,1,1,1)',
 'TUNIT7': 'K',
 'CTYPE1': 'FREQ-OBS',
 'CRVAL1': 1399816838.1210938,
 'CRPIX1': 16385.0,
 'CDELT1': -1525.87890625,
 'CTYPE2': 'RA',
 'CRVAL2': 206.85210757719534,
 'CTYPE3': 'DEC',
 'CRVAL3': -30.407015310885345,
 'CRVAL4': -6,
 'OBSERVER': 'Jeff Mangum',
 'OBSID': 'unknown',
 'SCAN': 51,
 'OBSMODE': 'OnOff:PSWITCHON:TPWCAL',
 'FRONTEND': 'Rcvr1_2',
 'TCAL': 1.4526499509811401,
 'VELDEF': 'OPTI-LSR',
 'VFRAME': 22609.232181632025,
 'RVSYS': 4376523.966595529,
 'OBSFREQ': 1399816838.1210938,
 'LST': 54470.49953792763,
 'AZIMUTH': 198.1588901992111,
 'ELEVATIO': 18.69449298449073,
 'TAMBIENT': 292.54999999999995,
 'PRESSURE': 697.7295162882529,
 'HUMIDITY': 0.754,
 'RESTFREQ': 1420405000.0,
 'FREQRES': 1846.3134765625,
 'EQUINOX': 2000.0,
 'RADESYS': 'FK5',
 'TRGTLONG': 206.85199999999995,
 'TRG

In [10]:
ta.meta["VELOCITY"] = 1000000.0
# ta.meta["VELDEF"] = "foobar"
ta.velocity
ta.radial_velocity   # this should be in plot ???    tied to VELDEF

<Quantity 4410.07003909 km / s>

In [11]:
ta.plot()

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


For the remainder of the notebook, we focus on the small section near 1.39 GHz where a random spike can be seen. After some trial and error this happens in channel 21921 where the peak value is 0.59789101 K.

explain more why

Lets plot this zoomed spectrum in km/s

In [12]:
if ta.nchan > 1000:
    print("Taking a zoomed spectrum")
    print(ta[21920:21923])   # print values around the little spike
    ta = ta[21850:22000]     # these are the 150 channels we will zoom into
else:
    print(f"Already had the zoomed spectrum of {ta.nchan} channels")
    print(ta[70:73])
ta.plot(xaxis_unit='km/s')
print(ta.velocity_axis_to("km/s")[71])     # 6256.475    velocity of the spike in this frame 

Taking a zoomed spectrum
Spectrum (length=3)
Flux=[0.33822484 0.59789101 0.26609857] K,  mean=0.40074 K
Spectral Axis=[1.39136957e+09 1.39136805e+09 1.39136652e+09] Hz,  mean=1391368046.61719 Hz


/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


6256.475163870091 km / s


## Writing a spectrum 

The default `Spectrum.write()` uses its native units (Hz) to write a spectrum. More useful is the spectral axis in km/s. One way is to make a copy of the spectrum while changing the units, using 
[Spectrum.with_spectral_axis_unit()](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum.html#specutils.Spectrum.with_spectral_axis_unit)

In [13]:
# should not use private functions.   See issue #299 and #527
# this also changed the spectral axis

#ta._spectral_axis = ta._spectral_axis.to("km/s") 
#ta.write("junk1v.tab",format="ascii.commented_header", overwrite=True)

In [14]:
ta.with_spectral_axis_unit?

Signature: ta.with_spectral_axis_unit(unit, velocity_convention=None, rest_value=None)
Docstring:
Returns a new spectrum with a different spectral axis unit. Note that this creates a new
object using the converted spectral axis and thus drops the original WCS, if it existed,
replacing it with a lookup-table :class:`~gwcs.wcs.WCS` based on the new spectral axis. The
original WCS will be stored in the ``original_wcs`` entry of the new object's ``meta``
dictionary.

Parameters
----------
unit : :class:`~astropy.units.Unit`
    Any valid spectral unit: velocity, (wave)length, or frequency.
    Only vacuum units are supported.
velocity_convention : 'relativistic', 'radio', or 'optical'
    The velocity convention to use for the output velocity axis.
    Required if the output type is velocity. This can be either one
    of the above strings, or an `astropy.units` equivalency.
rest_value : :class:`~astropy.units.Quantity`
    A rest wavelength or frequency with appropriate units.  Required i

In [15]:
# note cannot change frame
ta.set_frame("gcrs")
ta1 = ta.with_spectral_axis_unit("km/s",velocity_convention="radio", rest_value=1420*u.MHz)
#ta1.plot(xaxis_unit='km/s')
ta1

<Spectrum(flux=[0.34372934571614594 ... 0.30729395615058785] K (shape=(150,), mean=0.28094 K); spectral_axis=<SpectralAxis 
   (doppler_rest=1420.0 MHz
    doppler_convention=radio)
  [6021.84167018 6022.16381606 6022.48596194 ... 6069.19711481 6069.51926069
 6069.84140657] km / s> (length=150))>

In [16]:
ta1.write("ngc5291_spike.tab",format="ascii.commented_header", overwrite=True)

In [17]:
ta1

<Spectrum(flux=[0.34372934571614594 ... 0.30729395615058785] K (shape=(150,), mean=0.28094 K); spectral_axis=<SpectralAxis 
   (doppler_rest=1420.0 MHz
    doppler_convention=radio)
  [6021.84167018 6022.16381606 6022.48596194 ... 6069.19711481 6069.51926069
 6069.84140657] km / s> (length=150))>

In [18]:
# short cut to prevent something like ta1 to become
ta.with_frame("gcrs").with_spectral_axis_unit("km/s",velocity_convention="radio", rest_value=1420*u.MHz).write("ngc5291_spike.tab",format="ascii.commented_header", overwrite=True)

PJT

radial_velocity=4410.070039086893 km / s
      redshift=0.014820217767934851
doppler_rest=1420405000.0 Hz
    doppler_convention=optical)
    
      but 4410.070039/c = 0.0147104
      
and 0.014820217767934851*c = 4442.99


ok, see above

### Change Rest Frame
You can change the velocity frame by supplying one of the [built-in astropy coordinate frames](https://docs.astropy.org/en/stable/coordinates/index.html#built-in-frame-classes). These are specified by string name.

For example, to plot in the barycentric frame use `"icrs"`. The change from topocentric to barycantric is small, a shift of 22.8 kHz (PJT:  25 km/s or 120 kHz???)

In [19]:
ta.plot(vel_frame='icrs', xaxis_unit='km/s',grid=True)
print(ta.velocity_axis_to("km/s")[71])     # only in plot, the printed value is stil the TOPO centric

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


6256.365280719204 km / s


In [20]:
# PJT:   and then bary is not recognized, where topo is... feature request?
# a.plot(vel_frame='bary', xaxis_unit='km/s')

In addition to the `astropy` frame names, we also allow `'topo'` and `'topocentric'`.

In [21]:
#ta.plot(vel_frame='topo', xaxis_unit='km/s')    # cannot use topo

### Doppler Convention 
One can also change the Doppler convention between `radio`, `optical`, and `relativistic`.
Here we also change the x-axis to velocity units and use LSRK frame.

PJT: Note astropy claims it's "LSRK", but only "lsrk" works.       

LSRK is the class name, lsrk the astropy names it recognized

In [22]:
ta.plot(vel_frame='lsrk', doppler_convention='radio', xaxis_unit='km/s')
#ta.plot(vel_frame=LSRK, doppler_convention='radio', xaxis_unit='km/s')   # need th class name imported

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


Finally, if you plot velocity units on the x-axis with no `vel_frame` given, it will default to the 
frame decoded from the VELDEF keyword in the header if present.

In [23]:
ta.plot(xaxis_unit="km/s")

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


## Changing the Spectral Axis of the `Spectrum`.
There are two ways to accomplish this.  One returns a copy of the original Spectrum with the new spectral axis; 
the other changes the spectral axis in place.

The `with_frame()` and `with_velocity_convention()` are used when returning a copy

The `set_frame()` and `set_convention()` are used in place  [PJT: notice the asymmetric used of function names]  -> feature requesy

### A.   Return a copy of the spectrum using [`Spectrum.with_frame()`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.with_frame)

In [24]:
newspec = ta.with_frame('galactocentric')                              # OPTI-GAL, but the V value has not changed !!!
newspec.plot(xaxis_unit="km/s",doppler_convention='radio')
print(f"The new spectral axis frame is {newspec.velocity_frame}")

#
newspec.write("junk2.tab",format="ascii.commented_header", overwrite=True)
# 1392090698.772761 0.598990
# 1391368046.6171937 0.597891008466955 
print(newspec.velocity_axis_to("km/s")[71]) # 6097.621

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


The new spectral axis frame is galactocentric
6097.6216873631065 km / s


In [25]:
newspec2 = ta.with_frame('galactocentric').with_velocity_convention("radio")
#newspec2 = ta.with_velocity_convention("radio").with_frame('galactocentric')
newspec2.plot(xaxis_unit="km/s")
print(newspec2.velocity_axis_to("km/s")[71]) # 5976.071

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


5976.071520452817 km / s


One can see that the spectral axis of the new spectrum is different. About 722 kHz.

In [26]:
ta.spectral_axis - newspec.spectral_axis


<Quantity [-722115.73779154, -722114.94592667, -722114.15406203,
           -722113.36219716, -722112.57033229, -722111.77846766,
           -722110.98660278, -722110.19473791, -722109.40287304,
           -722108.61100841, -722107.81914353, -722107.02727866,
           -722106.23541403, -722105.44354916, -722104.65168428,
           -722103.85981941, -722103.06795478, -722102.27608991,
           -722101.48422503, -722100.6923604 , -722099.90049553,
           -722099.10863066, -722098.31676579, -722097.52490115,
           -722096.73303628, -722095.94117141, -722095.14930677,
           -722094.3574419 , -722093.56557703, -722092.7737124 ,
           -722091.98184752, -722091.18998265, -722090.39811778,
           -722089.60625315, -722088.81438828, -722088.0225234 ,
           -722087.23065877, -722086.4387939 , -722085.64692903,
           -722084.85506415, -722084.06319952, -722083.27133465,
           -722082.47946978, -722081.68760514, -722080.89574027,
           -722080.103875

### B.   Change the spectral axis in place using 
[Spectrum.set_frame()](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.set_frame)

PJT:   should use the same example as previous copy version

In [27]:
print(ta.velocity_axis_to("km/s")[71]) # 6256.475
sa = ta.spectral_axis
ta.set_frame('gcrs')
print(f"Changed spectral axis frame to  {ta.velocity_frame}")
print(ta.velocity_axis_to("km/s")[71]) # 6256.365

6256.365280719204 km / s
Changed spectral axis frame to  gcrs
6256.365280719204 km / s


In [28]:
ta

<Spectrum(flux=[0.34372934571614594 ... 0.30729395615058785] K (shape=(150,), mean=0.28094 K); spectral_axis=<SpectralAxis 
   (observer: <GCRS Coordinate (obstime=2005-06-27T02:05:58.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
                  (-3418483.37116048, -3651331.51587568, 3945691.32972075)
               (v_x, v_y, v_z) in km / s
                  (-4.3110922e-08, 9.51169059e-08, 6.55464828e-08)>
    target: <FK5 Coordinate (equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
                (206.85210758, -30.40701531, 1000000.)
             (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                (0., 0., 1000.)>
    observer to target (computed from above):
      radial_velocity=1023.9624022280859 km / s
      redshift=0.0034214239589733797
    doppler_rest=1420405000.0 Hz
    doppler_convention=optical)
  [1.39147688e+09 1.39147536e+09 1.39147383e+09 ... 1.39125258e+09
 1.39125105e+09 1.39124953e+09]

In [29]:
ta.plot(xaxis_unit="km/s")

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


Here the original the spectral axis has changed, albeit by a very small change.

500 Hz or 0.10 km/s

In [30]:
ta.spectral_axis - sa

<Quantity [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0.] Hz>

In [31]:
ta.set_convention('radio')

In [32]:
ta.plot(xaxis_unit="km/s")
print(ta.velocity_axis_to("km/s")[71]) # 6128.47

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


6128.470305969093 km / s


In [33]:
ta.target

<SkyCoord (FK5: equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
    (206.85210758, -30.40701531, 1000000.)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (0., 0., 1000.)>

In [34]:
ta.observer

<GCRS Coordinate (obstime=2005-06-27T02:05:58.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
    (-3418483.37116048, -3651331.51587568, 3945691.32972075)
 (v_x, v_y, v_z) in km / s
    (-4.3110922e-08, 9.51169059e-08, 6.55464828e-08)>

In [35]:
# ta.set_frame('itrs')    # itrs will fail

## Other useful functions

### Spectral Axis Conversion

Convert the spectral axis to any units, frame, and convention with [`Spectrum.velocity_axis_to`](https://dysh.readthedocs.io/en/latest/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.velocity_axis_to). dysh understands some common synonyms like 'heliocentric' for astropy's 'hcrs'.

This returns an array, does not modify the spectrum meta data

? PJT:   why is toframe=   and not  vel_frame=

HeliocentricTrueEcliptic didn't work, but astropy claims it's a frame;   we convert it to lower case?



In [36]:
ta.velocity_axis_to(unit="pc/Myr", toframe='heliocentric', doppler_convention='radio')

<SpectralAxis 
   (observer: <HCRS Coordinate (obstime=2005-06-27T02:05:58.000): (x, y, z) in m
                  (1.44835706e+10, -1.38895619e+11, -6.02107693e+10)
               (v_x, v_y, v_z) in km / s
                  (0., 0., 0.)>
    target: <FK5 Coordinate (equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
                (206.85210758, -30.40701531, 1000000.)
             (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                (0., 0., 1000.)>
    observer to target (computed from above):
      radial_velocity=1000.0052133653335 km / s
      redshift=0.0033412402540600183
    doppler_rest=1420405000.0 Hz
    doppler_convention=radio)
  [6220.27269497, 6220.60208987, 6220.93148476, 6221.26087966,
   6221.59027455, 6221.91966945, 6222.24906434, 6222.57845924,
   6222.90785413, 6223.23724902, 6223.56664392, 6223.89603881,
   6224.22543371, 6224.5548286 , 6224.8842235 , 6225.21361839,
   6225.54301328, 6225.87240818, 6226.20180307, 6226.53

In [37]:
# ta.velocity_axis_to(toframe='heliocentricTrueEcliptic', doppler_convention='radio')

In [38]:
ta.velocity_axis_to(unit="km/s")

<SpectralAxis 
   (observer: <GCRS Coordinate (obstime=2005-06-27T02:05:58.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
                  (-3418483.37116048, -3651331.51587568, 3945691.32972075)
               (v_x, v_y, v_z) in km / s
                  (-4.3110922e-08, 9.51169059e-08, 6.55464828e-08)>
    target: <FK5 Coordinate (equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
                (206.85210758, -30.40701531, 1000000.)
             (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                (0., 0., 1000.)>
    observer to target (computed from above):
      radial_velocity=1023.9624022280859 km / s
      redshift=0.0034214239589733797
    doppler_rest=1420405000.0 Hz
    doppler_convention=radio)
  [6105.60446996, 6105.92652398, 6106.24857801, 6106.57063204,
   6106.89268607, 6107.2147401 , 6107.53679413, 6107.85884815,
   6108.18090218, 6108.50295621, 6108.82501024, 6109.14706427,
   6109.4691183 , 61

In [39]:
ta.velocity_axis_to(unit="GHz")

<SpectralAxis 
   (observer: <GCRS Coordinate (obstime=2005-06-27T02:05:58.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
                  (-3418483.37116048, -3651331.51587568, 3945691.32972075)
               (v_x, v_y, v_z) in km / s
                  (-4.3110922e-08, 9.51169059e-08, 6.55464828e-08)>
    target: <FK5 Coordinate (equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
                (206.85210758, -30.40701531, 1000000.)
             (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                (0., 0., 1000.)>
    observer to target (computed from above):
      radial_velocity=1023.9624022280859 km / s
      redshift=0.0034214239589733797
    doppler_rest=1420405000.0 Hz
    doppler_convention=radio)
  [1.39147688, 1.39147536, 1.39147383, 1.39147231, 1.39147078, 1.39146925,
   1.39146773, 1.3914662 , 1.39146468, 1.39146315, 1.39146162, 1.3914601 ,
   1.39145857, 1.39145705, 1.39145552, 1.391454  , 1.391452

In [40]:
ta[70:73]

<Spectrum(flux=<Quantity [0.33822484, 0.59789101, 0.26609857] K> (shape=(3,), mean=0.40074 K); spectral_axis=<SpectralAxis 
   (observer: <ITRS Coordinate (obstime=2005-06-27T02:05:58.000, location=(0., 0., 0.) km): (x, y, z) in m
                  (882593.9465029, -4924896.36541728, 3943748.74743984)
               (v_x, v_y, v_z) in km / s
                  (0., 0., 0.)>
    target: <SkyCoord (FK5: equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
                (206.85210758, -30.40701531, 1000000.)
             (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                (0., 0., 1000.)>
    observer to target (computed from above):
      radial_velocity=1024.0700390868951 km / s
      redshift=0.00342178422957673
    doppler_rest=1420405000.0 Hz
    doppler_convention=radio)
  [1.39136957e+09 1.39136805e+09 1.39136652e+09] Hz> (length=3))>

In [41]:
ta[70:73].velocity_axis_to(unit="km/s").value[1]

6128.575742455789

### Spectral Shift

Shift a spectrum in place to a given radial velocity or redshift with 
[Spectrum.shift_spectrum_to()](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D.shift_spectrum_to)

In [42]:
print(ta.velocity_axis_to("km/s")[71])

6128.470305969093 km / s


In [43]:
print(f"before shift {ta.spectral_axis}")
#ta.shift_spectrum_to(radial_velocity=0*u.km/u.s)
#ta.shift_spectrum_to(radial_velocity=4410.070039086893*u.km/u.s)
ta.shift_spectrum_to(radial_velocity=2000*u.km/u.s)
print(f"after shift {ta.spectral_axis}")

before shift [1.39147688e+09 1.39147536e+09 1.39147383e+09 1.39147231e+09
 1.39147078e+09 1.39146925e+09 1.39146773e+09 1.39146620e+09
 1.39146468e+09 1.39146315e+09 1.39146162e+09 1.39146010e+09
 1.39145857e+09 1.39145705e+09 1.39145552e+09 1.39145400e+09
 1.39145247e+09 1.39145094e+09 1.39144942e+09 1.39144789e+09
 1.39144637e+09 1.39144484e+09 1.39144331e+09 1.39144179e+09
 1.39144026e+09 1.39143874e+09 1.39143721e+09 1.39143568e+09
 1.39143416e+09 1.39143263e+09 1.39143111e+09 1.39142958e+09
 1.39142806e+09 1.39142653e+09 1.39142500e+09 1.39142348e+09
 1.39142195e+09 1.39142043e+09 1.39141890e+09 1.39141737e+09
 1.39141585e+09 1.39141432e+09 1.39141280e+09 1.39141127e+09
 1.39140974e+09 1.39140822e+09 1.39140669e+09 1.39140517e+09
 1.39140364e+09 1.39140212e+09 1.39140059e+09 1.39139906e+09
 1.39139754e+09 1.39139601e+09 1.39139449e+09 1.39139296e+09
 1.39139143e+09 1.39138991e+09 1.39138838e+09 1.39138686e+09
 1.39138533e+09 1.39138380e+09 1.39138228e+09 1.39138075e+09
 1.39137923

In [44]:
ta

<Spectrum(flux=[0.34372934571614594 ... 0.30729395615058785] K (shape=(150,), mean=0.28094 K); spectral_axis=<SpectralAxis 
   (observer: <GCRS Coordinate (obstime=2005-06-27T02:05:58.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, m)
                  (226.88638534, 38.2681561, 6370771.57549705)
               (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                  (-33132.93080836, 103576.96454981, -4.73214035e-08)>
    target: <FK5 Coordinate (equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
                (206.85210758, -30.40701531, 1000000.)
             (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                (-7.31878192e-19, 2.05840741e-19, 1976.03759777)>
    observer to target (computed from above):
      radial_velocity=1999.9999999922754 km / s
      redshift=0.006693684108879161
    doppler_rest=1420405000.0 Hz
    doppler_convention=radio)
  [1.3869

Changing the redshift?
Here's some confusion, printing the spectrum it says:

doppler_rest  which is spectrum.rest_value   (say 1420 MHz)


PJT  roundoff I guess
old: 4410.070039086893  0.014820217767934851
new: 4410.070039086889  0.014820217767934851




In [45]:
print(ta.velocity_axis_to("km/s")[71])  # 6128.575765276228 

7083.025777571336 km / s


In [46]:
ta.plot(xaxis_unit="km/s")

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


In [47]:
ta

<Spectrum(flux=[0.34372934571614594 ... 0.30729395615058785] K (shape=(150,), mean=0.28094 K); spectral_axis=<SpectralAxis 
   (observer: <GCRS Coordinate (obstime=2005-06-27T02:05:58.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, m)
                  (226.88638534, 38.2681561, 6370771.57549705)
               (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                  (-33132.93080836, 103576.96454981, -4.73214035e-08)>
    target: <FK5 Coordinate (equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
                (206.85210758, -30.40701531, 1000000.)
             (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                (-7.31878192e-19, 2.05840741e-19, 1976.03759777)>
    observer to target (computed from above):
      radial_velocity=1999.9999999922754 km / s
      redshift=0.006693684108879161
    doppler_rest=1420405000.0 Hz
    doppler_convention=radio)
  [1.3869

In [48]:
# going back to the original --- fails with target/observer
#ta.set_frame('itrs')   # itrs   fails!!     icrs works
ta.set_frame('icrs')
ta.set_convention('optical')
ta.plot(xaxis_unit="km/s")
print(ta.velocity_axis_to("km/s")[71]) 

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


7229.880607132361 km / s


In [49]:
ta2 = Spectrum.fake_spectrum()
print(ta.observer)

<ICRS Coordinate: (x, y, z) in m
    (1.51192456e+10, -1.38756853e+11, -6.01691775e+10)
 (v_x, v_y, v_z) in km / s
    (0., 0., 0.)>


In [50]:
# Spectrum.fake_spectrum().set_redshift_to(0.1)
# Spectrum.fake_spectrum().set_radial_velocity_to( 4000 * u.km/u.s)

# ValueError: Cannot specify radial velocity or redshift if both target and observer are specified

### Spectral Axis in Wavelengths

The default is angstrom, use [Quantity.to](https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.to) to convert to other units.

In [51]:
ta.wavelength.to('cm')

<SpectralAxis 
   (observer: <ICRS Coordinate: (x, y, z) in m
                  (1.51192456e+10, -1.38756853e+11, -6.01691775e+10)
               (v_x, v_y, v_z) in km / s
                  (0., 0., 0.)>
    target: <FK5 Coordinate (equinox=J2000.000): (ra, dec, distance) in (deg, deg, kpc)
                (206.85210758, -30.40701531, 1000000.)
             (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                (-7.31878192e-19, 2.05840741e-19, 1976.03759777)>
    observer to target (computed from above):
      radial_velocity=1976.037597771912 km / s
      redshift=0.006613218790137987
    doppler_rest=1420405000.0 Hz
    doppler_convention=optical)
  [21.61344367, 21.61346737, 21.61349107, 21.61351477, 21.61353847,
   21.61356217, 21.61358587, 21.61360957, 21.61363328, 21.61365698,
   21.61368068, 21.61370438, 21.61372808, 21.61375178, 21.61377549,
   21.61379919, 21.61382289, 21.61384659, 21.61387029, 21.613894  ,
   21.6139177 , 21.6139414 , 21.6139

### Plot in Channels

Also, you can plot the x-axis in channel units

In [52]:
ta.plot(xaxis_unit='chan')

/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/teuben/GBT/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


## Final Stats

Finally, at the end we compute some statistics over a spectrum, merely as a checksum if the notebook is reproducable.

In [53]:
ta.stats() # rms 0.09109935 K on full, and 0.07586939 K on the 21850:22000 (150 channel) portion of the original

{'mean': <Quantity 0.28093977 K>,
 'median': <Quantity 0.28012519 K>,
 'rms': <Quantity 0.07586939 K>,
 'min': <Quantity 0.05986537 K>,
 'max': <Quantity 0.59789101 K>,
 'npt': 150,
 'nan': 0}

In [54]:
ta.nchan

150

In [55]:
1000 * u.Unit("km/s")



<Quantity 1000. km / s>